In [ ]:
!pip install requests
!pip install bs4
!pip install playwright
#!pip install --upgrade gspread #oauth2client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 12.8 MB/s eta 0:00:00


In [ ]:
!playwright install

163.5 MiB [] 0% 0.0s163.5 MiB [] 0% 398.2s163.5 MiB [] 0% 210.8s163.5 MiB [] 0% 178.7s163.5 MiB [] 0% 156.4s163.5 MiB [] 0% 61.0s163.5 MiB [] 0% 37.6s163.5 MiB [] 0% 28.8s163.5 MiB [] 0% 23.5s163.5 MiB [] 0% 19.6s163.5 MiB [] 1% 17.7s163.5 MiB [] 1% 16.3s163.5 MiB [] 1% 14.9s163.5 MiB [] 1% 14.2s163.5 MiB [] 1% 13.6s163.5 MiB [] 2% 13.2s163.5 MiB [] 2% 13.0s163.5 MiB [] 2% 12.6s163.5 MiB [] 2% 12.7s163.5 MiB [] 2% 12.0s163.5 MiB [] 2% 11.6s163.5 MiB [] 3% 10.6s163.5 MiB [] 3% 9.8s163.5 MiB [] 4% 9.6s163.5 MiB [] 4% 9.0s163.5 MiB [] 4% 8.4s163.5 MiB [] 5% 8.2s163.5 MiB [] 5% 8.1s163.5 MiB [] 5% 8.0s163.5 MiB [] 5% 8.2s163.5 MiB [] 5% 8.3s163.5 MiB [] 6% 8.4s163.5 MiB [] 6% 8.6s163.5 MiB [] 6% 8.7s163.5 MiB [] 6% 8.8s163.5 MiB [] 6% 8.9s163.5 MiB [] 6% 9.0s163.5 MiB [] 6% 9.1s163.5 MiB [] 6% 9.2s163.5 MiB [] 7% 9.0s163.5 MiB [] 7% 8.9s163.5 MiB [] 8% 8.8s163.5 MiB [] 8% 8.6s163.5 MiB [] 8% 8.5s163.5 MiB [] 9% 8.4s163.5 MiB [] 9% 8.3s163.5 MiB [] 9% 8.4s163.5 MiB [] 9% 8.3s163.5 MiB [] 9%

In [ ]:
# Tournaments to pull data from
tournaments_list = ["https://gol.gg/tournament/tournament-matchlist/LCK%20Cup%202025/"]

# Used to input the data on the sheet
# FILL IN
client_id = ""
client_secret = ""

# Set up the API scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Paste Json Credentials Here
# FILL IN
creds_json = {
  "type": "service_account",
  "project_id": "",
  "private_key_id": "",
  "private_key": ""
  "client_email": "",
  "client_id": "",
  "auth_uri": "",
  "token_uri": "",
  "auth_provider_x509_cert_url": "",
  "client_x509_cert_url": "",
  "universe_domain": "googleapis.com"
}


sheet_name = "MatchData"
past_matches_sheet_name = "PastMatches"
spreadsheet_doc_name = "Fantasy League of Legends - LCK Cup 2025 - 2"

# All stats so we can delete the ones we do not use later
# Stats to use
stats_list = ["Kills", "Deaths", "Assists", "CS", "Vision Score", "Total damage to Champion", "Time ccing others"]

all_stats_list = ['Role', 'Level', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS', "CS in Team's Jungle", 'CS in Enemy Jungle', 'CSM', 'Golds', 'GPM', 'GOLD%', 'Vision Score', 'Wards placed', 'Wards destroyed', 'Control Wards Purchased', 'Detector Wards Placed', 'VSPM', 'WPM', 'VWPM', 'WCPM', 'VS%', 'Total damage to Champion', 'Physical Damage', 'Magic Damage', 'True Damage', 'DPM', 'DMG%', 'K+A Per Minute', 'KP%', 'Solo kills', 'Double kills', 'Triple kills', 'Quadra kills', 'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15', 'Objectives Stolen', 'Damage dealt to turrets', 'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates', 'Damage self mitigated', 'Total Damage Shielded On Teammates', 'Time ccing others', 'Total Time CC Dealt', 'Total damage taken', 'Total Time Spent Dead', 'Consumables purchased', 'Items Purchased', 'Shutdown bounty collected', 'Shutdown bounty lost']
stats_to_delete = list(set(stats_list) ^ set(all_stats_list))
print(stats_to_delete)

# Count the number of matches each team has played in to set the week
global_team_match_counter = {}

# Check to ensure the match is actually over
first_to = 2

['GPM', 'Wards destroyed', 'K+A Per Minute', 'Penta kills', 'CSD@15', 'Magic Damage', 'Damage self mitigated', 'Shutdown bounty collected', 'Golds', 'GOLD%', 'VS%', 'Total Damage Shielded On Teammates', 'VWPM', 'WCPM', "CS in Team's Jungle", 'Items Purchased', 'Detector Wards Placed', 'DMG%', 'Quadra kills', 'Objectives Stolen', 'Total damage taken', 'Consumables purchased', 'Control Wards Purchased', 'GD@15', 'Wards placed', 'Damage dealt to turrets', 'Level', 'Shutdown bounty lost', 'Physical Damage', 'VSPM', 'Total Time CC Dealt', 'XPD@15', 'True Damage', 'Double kills', 'KP%', 'CS in Enemy Jungle', 'KDA', 'Role', 'CSM', 'WPM', 'LVLD@15', 'DPM', 'Solo kills', 'Triple kills', 'Total Time Spent Dead', 'Damage dealt to buildings', 'Total heal', 'Total Heals On Teammates']


In [ ]:
import asyncio
from playwright.async_api import async_playwright, Playwright
import nest_asyncio
from bs4 import BeautifulSoup
import re

# Allow async functions to run inside of an async environment. This is required because jupyter notebooks are async
# environments, and the playwright package forces us to use the async version because of this. However, to use the async
# package, we must have it within an async function. So, we are using nest_asyncio.
nest_asyncio.apply()

# Parameters: playwright: the playwright object to use
# tournaments: list of string urls to scrape for matches
# get_matches reads all urls in the tournaments list and returns the ids of those matches,
# along with the number of games in that match and both team names in a nested list
async def get_matches(playwright: Playwright, tournaments):
  # Set the playwright to firefow and open a new page
  firefox = playwright.firefox
  browser = await firefox.launch()
  page = await browser.new_page()

  # Initialize an empty list of game ids to return later
  game_ids = []

  # Loop through all tournament urls
  for tournament in tournaments:
    # Open the url for the tournament and wait for the page to load
    await page.goto(tournament)
    content = await page.content()
    await page.wait_for_selector("tbody")

    # Get the table's HTML content
    table_html = await page.inner_html("tbody")  # Replace 'table' with specific selector if needed

    # Use BeautifulSoup to parse the table
    soup = BeautifulSoup(table_html, "html.parser")
    rows = soup.find_all("tr")

    # Use regex to match the scores
    score_pattern = re.compile(r'(\d+) - (\d+)')
    link_pattern = re.compile(r'/(\d+)/')

    # Loop through all rows (matches) in the tournament page
    for row in rows:
      # Get the score, which looks like "2 - 1" or "0 - 2". Matches the regex pattern
      score = row.find(name="td", class_=["text-center", "footable-visible"], string=score_pattern)
      # If there's a score, the match has been played and we can get the data
      # otherwise, do nothing
      if score:
        # Look for the game id in the row
        link = row.find("a").get("href")
        game_id = int(link_pattern.search(link).group(1))

        # Get the outcome of the match (2 - 1) in numbers to get the number of games played
        numbers = score_pattern.search(score.text)
        num1, num2 = map(int, numbers.groups())

        if not max(num1, num2) >= first_to:
          continue

        # Get the team names. It doesn't matter who won, but we need both team names
        winning_team = row.find(name="td", class_=["text_victory"]).text
        losing_team = row.find(name="td", class_=["text_defeat"]).text

        # Add all this data to the game_ids to return
        game_ids.append([game_id, num1 + num2, winning_team, losing_team])

  # Reverse the list to make the oldest games go first
  game_ids.reverse()
  return game_ids



In [ ]:
import pandas as pd

# playwright is a playwright object
# match is a list of lists with this as the inner list pattern: [game_id, number_of_games, winning_team, losing_team]
# get_match_data() gets these parameters and returns a pandas dataframe with the data of the players as the columns
# and the stats defined in stats_list as the rows + the number of games played
async def get_match_data(playwright: Playwright, match, stats_to_delete):
  # Get firefix browser and open a new page
  firefox = playwright.firefox
  browser = await firefox.launch()
  page = await browser.new_page()

  # Create an empty dataframe to store the player data
  total_player_data_df = pd.DataFrame()

  # Loop through each game in the match. On the site, it's based on the first id, and adds one
  # for each game in the match
  for id in range(match[0], match[0] + match[1]):
    # Create the url and open it
    url = "https://gol.gg/game/stats/" + str(id) + "/page-fullstats/"
    await page.goto(url)

    # Wait for the table to load
    await page.wait_for_selector("table")

    # Get the table's HTML content
    table_html = await page.inner_html("table")

    # Use BeautifulSoup to parse the table
    soup = BeautifulSoup(table_html, "html.parser")
    rows = soup.find_all("tr")
    player_data_matrix = []

    # Loop through all rows except the first one. The first row contains the champions, and we don't need that data
    for i in range(1, len(rows)):
      # Get the data within the rows
      cols = rows[i].find_all(["td", "th"])
      data = [col.text.strip() for col in cols]

      # Add the data to the 2d list
      player_data_matrix.append(data)

    # Turn the list into a matrix
    game_df = pd.DataFrame(player_data_matrix)

    # Set the first row as column headers
    game_df.columns = game_df.iloc[0]

    # Drop the first row
    game_df = game_df[1:]

    # Make the indexes look nice
    game_df.reset_index(drop=True, inplace=True)
    game_df.set_index('Player', inplace=True)

    # Remove all stats we don't want
    game_df.drop(stats_to_delete, inplace=True)

    # Make the data into all integers
    game_df = game_df.astype(int)

    # If the dataframe is empty, set the dataframe to the one we just made. Otherwise, add them together
    if total_player_data_df.empty:
      total_player_data_df = game_df
    else:
      column_labels_order = total_player_data_df.columns.tolist()
      game_df = game_df[column_labels_order]
      total_player_data_df += game_df
  # Add a bottom row for the number of games
  total_player_data_df.loc["Number of Games"] = [match[1] for i in range(total_player_data_df.shape[1])]

  # Make the data into all integers so it does not become a percentage on google sheets for some reason
  total_player_data_df = total_player_data_df.astype("int32")

  # Add another row on the bottom to show which team each player is on
  total_player_data_df.loc["Team"] = ["" for i in range(total_player_data_df.shape[1])]

  # Get the players on each team
  # Wait for this page to load
  url = "https://gol.gg/game/stats/" + str(match[0]) + "/page-summary/"
  await page.goto(url)
  await page.wait_for_selector("table")

  # Get the text of the headers and pick out the team names.
  h1_texts = await page.locator('div.col-cadre h1').all_text_contents()
  team1 = h1_texts[0]
  team2 = h1_texts[2]
  team_names = [team1, team2]

  # Get the tables with the player names to match the teams to the players
  tables = page.locator('table')

  # Get the number of tables found
  table_count = await tables.count()

  # Loop through all tables and extract their text content or HTML
  for i in range(table_count):
    # Get the table html
    table_html = await tables.nth(i).inner_html()
    soup = BeautifulSoup(table_html, "html.parser")

    # Get the names within the table
    names = soup.find_all("a")
    for j in range(len(names)):
      # Add the names to the players' column
      print(names[j].text, team_names[i])
      total_player_data_df.loc["Team", names[j].text] = team_names[i]
  return total_player_data_df


In [ ]:
print(global_team_match_counter)

{}


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import APIError
from gspread.utils import rowcol_to_a1

# parameter match_df is the dataframe of the match to upload to google sheets
# update_sheet sends the data in the dataframe to google sheets
def update_sheet(match_df, spreadsheet_doc_name, json_creds_dict):
  # Path to the service account JSON file
  credentials = ServiceAccountCredentials.from_json_keyfile_dict(json_creds_dict, scope)

  # Authenticate and connect to Google Sheets
  gc = gspread.authorize(credentials)

  # Open the Google Sheet by name
  spreadsheet = gc.open(spreadsheet_doc_name)

  # Select a specific worksheet within the document
  worksheet = spreadsheet.worksheet(sheet_name)

  # Turn the top row of player names into a list
  player_names_list = match_df.columns.tolist()

  # Loop through the 10 players in the dataframe
  for i in range(match_df.shape[1]):
    # Turn the player's data into a vertical list
    player_data = match_df[player_names_list[i]].values.reshape(-1, 1).tolist()

    # Remove the last element (the team name)
    team_name = player_data.pop()[0]
    print(player_names_list[i])
    print(player_data)

    # Find the column of the player's name in the spreadsheet
    player_cell_col = worksheet.find(player_names_list[i])

    # If it does not exist already, add it to the list
    if not player_cell_col:
      print("Name does not exist in the sheet.")
      num_columns = len(worksheet.row_values(1))

      # Add a column if we are running out. There's a weird quirk with insert_cols, where we can't insert at
      # the end of the sheet, so we insert close to the end and do it before we reach the end, so we always have about a
      # 2 column padding
      if num_columns > 23:
        worksheet.insert_cols([[]], col=num_columns - 1)
        num_columns += 1
      # Add the player's name to the sheet and get the column
      worksheet.update_cell(1, num_columns + 1, player_names_list[i])
      player_cell_col = worksheet.find(player_names_list[i])

    # Get the column in terms of google sheets columns rather than a number
    player_cell_col = rowcol_to_a1(1, player_cell_col.col).split('1')[0]

    # Get the row of the match number
    stat_cell_row = worksheet.find("Match " + str(global_team_match_counter[team_name]) + " " + stats_list[0]).row

    # Get the range to update and turn it into a google sheets range
    cell_to_update_top = player_cell_col + str(stat_cell_row)
    cell_to_update_bottom = player_cell_col + str(stat_cell_row + len(player_data))
    update_range = cell_to_update_top + ":" + cell_to_update_bottom

    # Update the sheet
    worksheet.update(player_data, update_range)

In [ ]:
# match is a parameter with the match id, number of games, winning team, and losing team in a list
# update_num_matches adds the team name to the dictionary if it is not there already, and it increments
# the number if it is there
def update_num_matches(match):
  if match[2] in global_team_match_counter:
    global_team_match_counter[match[2]] += 1
  else:
    global_team_match_counter[match[2]] = 1

  if match[3] in global_team_match_counter:
    global_team_match_counter[match[3]] += 1
  else:
    global_team_match_counter[match[3]] = 1

In [ ]:
import json
from datetime import date

# Gets all match ids of scraped matches
"""
def get_past_matches(path):
  with open(path_to_past_matches, "r") as f:
    past_matches = json.load(f)
  return past_matches

# Writes a list to the json file
def write_past_matches(path, matches):
  with open(path_to_past_matches, "w") as f:
    json.dump(matches, f)
"""
def get_past_matches(spreadsheet_doc_name, past_matches_sheet_name, json_creds_dict, scope):
    # Path to the service account JSON file
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(json_creds_dict, scope)

    # Authenticate and connect to Google Sheets
    gc = gspread.authorize(credentials)

    # Open the Google Sheet by name
    spreadsheet = gc.open(spreadsheet_doc_name)

    # Select a specific worksheet
    worksheet = spreadsheet.worksheet(past_matches_sheet_name)

    # Arg 1 is the first column
    column_data = worksheet.col_values(1)[1:]
    column_data = list(map(int, column_data))

    return column_data

def write_past_matches(spreadsheet_doc_name, past_matches_sheet_name, json_creds_dict, match_id, match_dataframe, scope):
    # Path to the service account JSON file
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(json_creds_dict, scope)

    # Authenticate and connect to Google Sheets
    gc = gspread.authorize(credentials)

    # Open the Google Sheet by name
    spreadsheet = gc.open(spreadsheet_doc_name)

    # Select a specific worksheet
    worksheet = spreadsheet.worksheet(past_matches_sheet_name)

    # Remove the duplicates
    teams = list(set(match_dataframe.loc["Team"]))
    team_1_string = teams[0] + "'s Match " + str(global_team_match_counter[teams[0]])
    team_2_string = teams[1] + "'s Match " + str(global_team_match_counter[teams[1]])

    today = date.today()
    formatted_date = today.strftime("%d/%m/%Y")

    list_to_append = [match_id, team_1_string, team_2_string, formatted_date]


    # Append the row at the last row
    worksheet.append_row(list_to_append)

In [ ]:
#write_past_matches(spreadsheet_doc_name, past_matches_sheet_name, creds_json, )
print(get_past_matches(spreadsheet_doc_name, past_matches_sheet_name, creds_json, scope))

NameError: name 'spreadsheet_doc_name' is not defined

In [ ]:
import time

# Runs the methods
async def main():
  # Reset the dictionary to make sure the match numbers are correct
  global_team_match_counter.clear()

  # Scrape the page for all matches
  async with async_playwright() as playwright:
    all_matches = await get_matches(playwright, tournaments_list)

  past_matches = get_past_matches(spreadsheet_doc_name, past_matches_sheet_name, creds_json, scope)
  # Loop through all matches
  for league_match in all_matches:
    # Increment the number of matches the team has played
    update_num_matches(league_match)

    # Get the past matches from the json file to see if this match has been scraped already

    # If it has not been scraped already, scrape it. Otherwise, skip it
    if not league_match[0] in past_matches:
      # Scrape the webpage and get the data as a dataframe
      async with async_playwright() as playwright:
        match_df = await get_match_data(playwright, league_match, stats_to_delete)

      # Use the data to update the sheet
      update_sheet(match_df, spreadsheet_doc_name, creds_json)

      # Add the match we scraped to the json file for permanent storage
      past_matches.append(league_match[0])
      write_past_matches(spreadsheet_doc_name, past_matches_sheet_name, creds_json, league_match[0], match_df, scope)

      # Wait 1 minute so we don't make too many requests to the sheets api and get an error
      time.sleep(35)
    else:
      print("Match Skipped: " + str(league_match[0]))


In [ ]:
asyncio.run(main())

Match Skipped: 62967
Match Skipped: 62970
Match Skipped: 63017
Match Skipped: 63020
Match Skipped: 63023
Match Skipped: 63026
Match Skipped: 63029
Match Skipped: 63032
Match Skipped: 63035
Match Skipped: 63038
Match Skipped: 63158
Match Skipped: 63161
Match Skipped: 63164
Match Skipped: 63167
Match Skipped: 63170
Match Skipped: 63173
Match Skipped: 63176
Match Skipped: 63179
Match Skipped: 63182
Match Skipped: 63185
DuDu DN Freecs
Pyosik DN Freecs
Bulldog DN Freecs
Berserker DN Freecs
Life DN Freecs
PerfecT KT Rolster
Cuzz KT Rolster
Bdd KT Rolster
deokdam KT Rolster
Way KT Rolster
DuDu
[[8], [7], [9], [807], [85], [38873], [47], [3]]
Pyosik
[[2], [7], [16], [589], [151], [22457], [45], [3]]
Bulldog
[[5], [5], [10], [828], [99], [57255], [61], [3]]
Berserker
[[7], [4], [10], [945], [118], [47637], [20], [3]]
Life
[[0], [10], [18], [103], [350], [16973], [78], [3]]
PerfecT
[[5], [3], [15], [806], [105], [40981], [59], [3]]
Cuzz
[[9], [6], [16], [738], [150], [41492], [41], [3]]
Bdd
[[6]

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import APIError
from gspread.utils import rowcol_to_a1

# This is for manually filling in the column in the data sheet. Just fills the leftmost column with the stats we want it to have
def fill_data_column(num_matches, json_creds_dict, scope):
  # Path to the service account JSON file
  credentials = ServiceAccountCredentials.from_json_keyfile_dict(json_creds_dict, scope)

  # Authenticate and connect to Google Sheets
  gc = gspread.authorize(credentials)

  # Open the Google Sheet by name
  spreadsheet = gc.open(spreadsheet_doc_name)

  # Select a specific worksheet
  worksheet = spreadsheet.worksheet("TestSheet")

  column = []
  for i in range(1, num_matches + 1):
    for j in range(len(stats_list)):
      column.append(["Match " + str(i) + " " + stats_list[j]])
      if j == len(stats_list) - 1:
        column.append(["Match " + str(i) + " Number of Games"])
  print(column)
  update_range = "A2:A"
  worksheet.update(column, update_range)

In [ ]:
fill_data_column(2, creds_json, scope)

[['Match 1 Kills'], ['Match 1 Deaths'], ['Match 1 Assists'], ['Match 1 CS'], ['Match 1 Vision Score'], ['Match 1 Total damage to Champion'], ['Match 1 Time ccing others'], ['Match 1 Number of Games'], ['Match 2 Kills'], ['Match 2 Deaths'], ['Match 2 Assists'], ['Match 2 CS'], ['Match 2 Vision Score'], ['Match 2 Total damage to Champion'], ['Match 2 Time ccing others'], ['Match 2 Number of Games']]


In [ ]:
#write_past_matches(path_to_past_matches, [])

In [ ]:
#past = get_past_matches(path_to_past_matches)
#print(past)
#past.pop()
#past = past[2:]
#print(past)
#write_past_matches(path_to_past_matches, past)